In [105]:
import os
import sys
sys.path.append(os.path.dirname((os.path.abspath(''))))
import pandas as pd

from src.pipeline import Pipeline
from src.preprocessing.preprocessing import tokenization, removal, stemming

In [132]:
collection = pd.read_csv('../data/TREC_Passage/collection.tsv', sep='\t', names=["pID", "Passage"], header=None)
queries = pd.read_csv('../data/TREC_Passage/queries.dev.tsv', sep='\t', names=["qID", "Query"], header=None)

In [ ]:
#pipeline = Pipeline(collection = '../data/raw/collection.pkl', queries = '../data/raw/queries.pkl')
#pipeline = Pipeline(collection = collection, queries = queries)

In [133]:
collection.Passage

0          The presence of communication amid scientific ...
1          The Manhattan Project and its atomic bomb help...
2          Essay on The Manhattan Project - The Manhattan...
3          The Manhattan Project was the name for a proje...
4          versions of each volume as well as complementa...
                                 ...                        
8841818    When metal salts emit short wavelengths of vis...
8841819    Thousands of people across the United States w...
8841820    The recipe that creates blue, for example, inc...
8841821    On Independence Days of yore, old-timey crowds...
8841822    View full size image. Behind the scenes of the...
Name: Passage, Length: 8841823, dtype: object

In [103]:
queries.head()

,qID,Query
0,1048578,cost of endless pools/swim spa
1,1048579,what is pcnt
2,1048580,what is pcb waste
3,1048581,what is pbis?
4,1048582,what is paysky


In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from tqdm import tqdm
import numpy as np


class TFIDF(object):

    vectorizer = None

    def fit(self, text_in_tokens: pd.Series):
        def dummy(text):
            return text

        self.vectorizer = TfidfVectorizer(tokenizer=dummy, lowercase=False)
        self.vectorizer.fit(text_in_tokens)

        return self

    def transform(self, text_in_tokens: pd.Series):
        assert self.vectorizer is not None, 'You need to fit me first'

        tfidf_matrix = self.vectorizer.transform(text_in_tokens)
        token_names = self.vectorizer.get_feature_names()
        tf_idf_dict = {}
        j = 0

        for name in token_names:
            tf_idf_dict[name] = j
            j += 1

        tf_idf_list = []
        for i in tqdm(range(len(text_in_tokens))):
            tf_idf_token = {}
            for token in text_in_tokens[i]:
                tf_idf_token[token] = tfidf_matrix[i, tf_idf_dict[token]]
            tf_idf_list.append(tf_idf_token)
        tf_idf_vec = np.array(tf_idf_list)

        return tf_idf_vec


In [134]:
x = collection.Passage[:40]
x

0     The presence of communication amid scientific ...
1     The Manhattan Project and its atomic bomb help...
2     Essay on The Manhattan Project - The Manhattan...
3     The Manhattan Project was the name for a proje...
4     versions of each volume as well as complementa...
5     The Manhattan Project. This once classified ph...
6     Nor will it attempt to substitute for the extr...
7     Manhattan Project. The Manhattan Project was a...
8     In June 1942, the United States Army Corps of ...
9     One of the main reasons Hanford was selected a...
10    group discussions, community boards or panels ...
11    punishment designed to repair the damage done ...
12    Tutorial: Introduction to Restorative Justice....
13    Organize volunteer community panels, boards, o...
14    The purpose of this paper is to point out a nu...
15    Each of these types of communitiesâthe geogr...
16    The approach is based on a theory of justice t...
17    Inherent in many peopleâs understanding 

In [136]:
x = x.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))))

100%|██████████| 40/40 [00:00<00:00, 350.57it/s]


In [139]:
#collection['tfidf_embedding'] = tfidf.fit(x).transform(x)
output = tfidf.fit(x).transform(x)

c:\users\d073999\documents\priv-dev\retriever\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
c:\users\d073999\documents\priv-dev\retriever\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
100%|██████████| 40/40 [00:00<00:00, 1045.88it/s]

{'presenc': 0.19169979611672686}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537, 'mind': 0.19169979611672686}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537, 'mind': 0.19169979611672686, 'equal': 0.19169979611672686}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537, 'mind': 0.19169979611672686, 'equal': 0.19169979611672686, 'import': 0.19169979611672686}
{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537, 'mind':

In [142]:
output

array([{'presenc': 0.19169979611672686, 'commun': 0.11998316454888718, 'amid': 0.19169979611672686, 'scientif': 0.3833995922334537, 'mind': 0.19169979611672686, 'equal': 0.19169979611672686, 'import': 0.19169979611672686, 'success': 0.34473324190582777, 'manhattan': 0.11041488942969002, 'project': 0.11041488942969002, 'intellect': 0.19169979611672686, 'cloud': 0.19169979611672686, 'hang': 0.19169979611672686, 'impress': 0.19169979611672686, 'achiev': 0.19169979611672686, 'atom': 0.12559923363629932, 'research': 0.17236662095291388, 'engin': 0.15864951487651308, 'truli': 0.19169979611672686, 'meant': 0.19169979611672686, 'hundr': 0.19169979611672686, 'thousand': 0.19169979611672686, 'innoc': 0.19169979611672686, 'live': 0.19169979611672686, 'obliter': 0.19169979611672686},
       {'manhattan': 0.15059747631090326, 'project': 0.15059747631090326, 'atom': 0.3426155242270081, 'bomb': 0.17130776211350404, 'help': 0.2614638809458508, 'bring': 0.23509490658210916, 'end': 0.21638582152967742, 

In [ ]:
!pip install "tensorflow>=2.0.0"
!pip install --upgrade tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:

elmo = elmo_module()

In [ ]:
!pip install allennlp==0.9.0

In [ ]:
!pip install flair

In [98]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence

# init embedding
embedding = ELMoEmbeddings()

In [99]:
# embed words in sentence
# create a sentence
sentence = Sentence('The grass is green .')

# embed words in sentence
embedding.embed(sentence)
for token in sentence:
    print(token)
    print(token.embedding)

Token[0]: "The"
tensor([-0.3288,  0.2022, -0.5940,  ..., -1.2773,  0.3049,  0.2150])
Token[1]: "grass"
tensor([ 0.2539, -0.2363,  0.5263,  ..., -0.7001,  0.8798,  1.4191])
Token[2]: "is"
tensor([ 0.1915,  0.2300, -0.2894,  ..., -0.3626,  1.9066,  1.4520])
Token[3]: "green"
tensor([ 0.1779,  0.1309, -0.1041,  ..., -0.1006,  1.6152,  0.3299])
Token[4]: "."
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.0106, -0.0833,  0.0669])


In [144]:
x = collection.Passage[:100]
#sentence_list = [embedding.embed(Sentence(line)) for line in x]

In [145]:
for line in x:
    sentence = Sentence(line)
    emb = embedding.embed(sentence)
    for token in sentence:
        print(token.embedding)

tensor([-0.3288,  0.2022, -0.5940,  ..., -0.2879,  0.4912,  0.0945])
tensor([-0.4335,  1.0174, -0.7131,  ..., -0.3751,  1.4165, -0.4075])
tensor([ 0.2478, -0.2490,  0.0222,  ..., -1.6505, -0.4502, -0.1458])
tensor([ 0.3358, -0.6698, -0.0683,  ...,  0.5426,  0.2744, -0.2716])
tensor([-5.6628e-01, -2.1065e-02,  2.1309e-06,  ..., -4.3523e-01,
         7.5148e-01, -5.5541e-01])
tensor([-0.3897, -0.5744,  0.1275,  ...,  1.6140,  0.3423, -1.2748])
tensor([-0.4204, -0.1053, -0.7463,  ...,  1.6170,  1.7666, -0.2729])
tensor([ 0.2981, -0.0738, -0.1319,  ...,  0.3481,  0.7208,  0.2200])
tensor([-0.1151, -0.6863, -0.0091,  ...,  0.2669,  0.0373, -1.0258])
tensor([-0.4031, -0.2223, -0.0422,  ..., -0.1549,  0.9292, -1.7620])
tensor([-0.1821,  0.0381,  0.1691,  ...,  0.5510, -0.8244, -1.1586])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.5522, -0.1269, -1.0468])
tensor([-0.5432, -0.2020, -0.1650,  ..., -0.9970,  2.7737, -0.8996])
tensor([ 0.2478, -0.2490,  0.0222,  ..., -1.7679,  0.1539, -1.2161])
te

tensor([-0.3288,  0.2022, -0.5940,  ..., -0.5849,  0.5399, -0.5090])
tensor([-0.1881, -0.2484,  0.5997,  ..., -0.7757,  1.5385, -0.4913])
tensor([ 0.2678, -0.2839, -1.6761,  ..., -0.0675,  1.7258, -0.5118])
tensor([ 0.2981, -0.0738, -0.1319,  ..., -1.3374,  0.3530,  0.1457])
tensor([-0.0690,  0.1126,  0.2371,  ..., -0.7484,  0.0992,  0.2822])
tensor([-0.2674, -0.1357, -0.1691,  ..., -0.1449,  1.6824,  0.4698])
tensor([-0.2415,  0.0541, -0.3114,  ..., -0.1964, -0.2709, -0.3315])
tensor([ 0.1040,  0.1229, -0.0706,  ...,  0.0404,  0.7718, -0.6315])
tensor([ 0.2009, -0.1494, -0.7966,  ..., -0.8937,  1.5359, -0.1754])
tensor([ 0.4359, -0.2371,  0.1499,  ..., -0.6557,  0.5641,  0.1783])
tensor([-0.3088,  0.6863,  0.3279,  ..., -1.4629, -0.0313, -0.7643])
tensor([ 1.1053, -0.1148, -0.2128,  ...,  0.2435,  0.9960, -0.3165])
tensor([-0.2445, -0.7706, -0.9120,  ...,  1.5802, -0.2082, -2.2484])
tensor([ 0.2945, -0.0066,  0.2763,  ...,  0.1552,  0.8685, -1.0772])
tensor([ 0.1131,  0.1575,  0.1414,

tensor([ 0.8518, -0.9071, -1.4499,  ..., -0.1968,  0.1873, -0.5878])
tensor([ 0.3605, -0.1958,  0.0624,  ..., -0.9928,  0.5103, -0.1919])
tensor([ 0.1039, -0.4565,  0.0605,  ...,  0.5889, -0.4781, -1.5301])
tensor([ 1.3591, -0.1410, -0.3595,  ...,  1.1604,  1.2490, -0.2488])
tensor([-0.1821,  0.0381,  0.1691,  ...,  1.0424,  1.2525, -1.2010])
tensor([-0.3701,  1.1267,  1.6215,  ...,  0.9781,  1.4080, -0.5670])
tensor([-0.2415,  0.0541, -0.3114,  ...,  0.2537, -0.2949,  0.0986])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.0495,  0.7729, -0.7797])
tensor([ 0.4271,  0.1026,  0.3017,  ..., -0.2192,  1.2059,  0.2509])
tensor([-0.0893, -0.4715, -0.7548,  ..., -0.3874,  1.9130, -0.5590])
tensor([-0.4412, -0.9116,  0.4660,  ...,  0.7231,  1.0878, -0.1878])
tensor([ 0.0434,  0.1392, -0.4155,  ..., -2.2781, -0.9842, -0.8171])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.2261, -0.1708, -0.2579])
tensor([-0.0623,  1.1052, -0.1352,  ...,  1.6196,  0.2126, -0.1868])
tensor([-0.0774, -0.3940, -0.3256,

tensor([-0.2053,  0.2502, -0.7327,  ..., -0.4341,  0.1708, -0.2841])
tensor([ 7.2008e-01,  2.3552e-01, -2.3989e-04,  ...,  2.8968e-01,
         1.4794e+00, -1.4140e+00])
tensor([-0.5423,  0.0268, -0.2013,  ...,  0.9124,  1.7303, -1.0152])
tensor([0.1131, 0.1575, 0.1414,  ..., 0.1662, 0.6187, 0.3036])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.1045,  0.2069, -0.8764])
tensor([-0.3252, -0.4169,  0.3226,  ..., -0.5232,  0.2941, -0.0987])
tensor([ 0.2210, -0.2911,  0.3106,  ...,  1.6541,  1.6955, -1.0435])
tensor([ 0.7099, -0.4779,  0.0939,  ...,  0.2354,  0.2434,  0.2262])
tensor([ 1.2369, -1.2607, -0.4261,  ...,  0.9797,  0.3033, -0.2469])
tensor([ 0.2478, -0.2490,  0.0222,  ..., -1.8486,  1.0444, -0.4696])
tensor([-0.4502,  0.5195,  0.1148,  ...,  0.3875,  0.4570, -0.7563])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.0095,  0.3441, -0.5224])
tensor([-0.1881, -0.2484,  0.5997,  ..., -0.4356,  1.6331, -0.6439])
tensor([ 0.2678, -0.2839, -1.6761,  ...,  0.0458,  2.6724,  0.5413])
tensor([

tensor([ 0.0260,  0.0271,  0.7579,  ...,  0.7305, -0.3670, -0.9243])
tensor([-0.9187,  0.5547, -0.5026,  ..., -0.2354, -1.0852, -0.7231])
tensor([ 3.7346e-04,  2.3383e-01, -5.5994e-01,  ...,  6.2173e-01,
         7.1350e-01, -1.5085e-01])
tensor([-0.1821,  0.0381,  0.1691,  ...,  0.4405, -0.8735,  0.2582])
tensor([-1.6261, -0.4007,  0.5780,  ...,  1.2136, -0.8311, -1.4939])
tensor([-0.4078, -0.7021,  0.3506,  ...,  1.2488, -1.4081, -1.4521])
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.6517, -1.0355, -0.0264])
tensor([-1.6261, -0.4007,  0.5780,  ...,  0.0415, -1.0180, -0.3873])
tensor([-0.4340, -0.7386,  1.0174,  ...,  0.9213, -0.7740, -1.0660])
tensor([ 0.1915,  0.2300, -0.2894,  ..., -1.7971, -1.0504,  0.8404])
tensor([ 0.1040,  0.1229, -0.0706,  ...,  0.0410,  0.2589,  0.2441])
tensor([-0.3040, -0.0634, -1.2906,  ..., -1.0214,  0.8555, -0.1483])
tensor([ 0.2478, -0.2490,  0.0222,  ..., -2.2627, -0.6239,  0.4303])
tensor([-0.4340, -0.7386,  1.0174,  ...,  0.9827, -0.2206, -1.2113])
te

...

tensor([-0.2901,  0.3624, -0.5452,  ..., -0.5905, -0.3411,  1.4320])
tensor([0.1131, 0.1575, 0.1414,  ..., 0.3427, 0.2253, 1.3976])
tensor([ 0.3645, -0.1415, -0.0662,  ..., -0.9410,  0.2253,  1.0691])
tensor([-0.4262,  0.3978,  0.1398,  ...,  0.7263,  1.0331,  0.9767])
tensor([0.0218, 0.5739, 0.7175,  ..., 0.4981, 0.3377, 0.7792])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.8340, -0.7218,  1.1035])
tensor([-0.1736,  0.0736, -0.3008,  ...,  0.3013,  0.0214,  1.8501])
tensor([-1.3051,  0.5029, -0.7726,  ..., -0.5865, -0.7294,  2.3745])
tensor([-0.2543,  0.0933,  0.0835,  ..., -0.8040, -0.2998,  2.4617])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.8458, -0.1725,  1.2627])
tensor([-0.0770,  0.6666,  0.3972,  ..., -0.4477, -0.3808,  2.4671])
tensor([ 0.1366,  1.1099,  0.5916,  ..., -0.7664, -0.7274,  1.3373])
tensor([ 0.2478, -0.2490,  0.0222,  ..., -3.1873, -0.1343,  1.6398])
tensor([-0.0690,  0.1126,  0.2371,  ...,  1.2659,  0.3545,  1.2637])
tensor([ 0.3661, -0.7175,  0.9417,  ...,  0.51

tensor([-0.0159, -0.0344, -0.0616,  ...,  0.5229,  0.0698, -1.1816])
tensor([-0.0238, -0.0811,  0.1039,  ..., -0.0757,  0.6619, -1.5232])
tensor([-1.2289,  0.2348, -0.1960,  ...,  0.7100,  0.9683, -2.0780])
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.5492,  0.2076, -0.4669])
tensor([-0.4350,  0.2461,  0.0982,  ..., -1.2296,  0.4448, -0.2637])
tensor([-0.4353,  0.2586,  0.3547,  ...,  1.0958,  2.5637, -1.1740])
tensor([ 0.1131,  0.1575,  0.1414,  ..., -0.5042,  1.2363, -0.2382])
tensor([-0.0105, -0.5830,  0.2711,  ..., -0.4403, -0.0462, -0.8161])
tensor([-0.4959,  0.2016,  0.2681,  ...,  0.6225, -1.9498, -1.3209])
tensor([ 0.1131,  0.1575,  0.1414,  ...,  0.1852,  0.4948, -0.0816])
tensor([ 0.2313, -0.3767, -0.6084,  ..., -0.7144,  0.3791, -0.3429])
tensor([-0.0980,  0.2265,  0.1585,  ...,  0.0830,  1.4878, -0.8802])
tensor([-0.7524, -0.4685,  0.1846,  ...,  0.8982,  0.0936, -1.5773])
tensor([ 0.1979, -1.0170,  0.4579,  ...,  0.0027, -0.1816, -1.8269])
tensor([-0.4899, -0.0432,  0.1969,

tensor([ 0.1833,  0.1160, -0.1243,  ..., -0.4033, -1.1809,  0.0525])
tensor([-0.0105, -0.5830,  0.2711,  ..., -0.6324, -0.4206,  0.0884])
tensor([ 1.1961,  0.5340,  0.6493,  ..., -1.0153,  0.0580, -1.0055])
tensor([-0.2543,  0.0933,  0.0835,  ..., -2.3767, -0.9533,  0.3400])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.3021, -0.4895, -1.4550])
tensor([-0.5611,  0.5039,  0.3670,  ..., -0.6224, -1.1810, -0.9536])
tensor([-0.2587, -0.3169,  0.6655,  ...,  0.2003, -1.6287, -0.7251])
tensor([ 1.1177, -0.8388,  0.5548,  ..., -1.2865,  0.3421, -1.1394])
tensor([0.1131, 0.1575, 0.1414,  ..., 0.8547, 0.1704, 0.3480])
tensor([-0.4644, -0.9879,  0.4039,  ...,  0.4993,  0.8766, -0.4992])
tensor([ 0.1131,  0.1575,  0.1414,  ..., -0.2588,  1.0095,  0.8296])
tensor([-0.3235,  0.1270,  0.2056,  ..., -0.0246,  0.0581, -0.3597])
tensor([-0.1115, -0.3943, -0.3937,  ..., -0.4339, -0.0466,  0.6164])
tensor([-0.0233, -0.0715, -0.8086,  ..., -0.4400,  1.1993,  0.5795])
tensor([-0.9187,  0.5547, -0.5026,  ...,

tensor([-0.0276, -0.4785, -0.6906,  ..., -0.0605,  0.0753,  0.8073])
tensor([ 0.6731, -0.2125,  0.3066,  ...,  1.1537,  1.6911, -0.2580])
tensor([ 0.1040,  0.1229, -0.0706,  ...,  1.8547,  1.1476, -0.5975])
tensor([ 0.9795,  0.2540, -1.0375,  ...,  0.0782,  1.3312, -0.1910])
tensor([ 0.2478, -0.2490,  0.0222,  ...,  3.0635, -1.2495, -0.2858])
tensor([ 1.6059, -0.3057, -0.3803,  ...,  1.8774,  0.5094,  0.4737])
tensor([ 1.2441,  0.2328, -0.0104,  ...,  1.0255,  0.0402, -0.6056])
tensor([-0.2569,  0.4148, -0.4112,  ..., -1.7184, -0.4583, -0.4209])
tensor([-0.3706,  0.5851,  0.2687,  ...,  0.8610,  1.2176,  0.0077])
tensor([-0.0979, -0.7214, -0.5683,  ...,  0.5734,  0.9835, -0.6847])
tensor([-0.0248, -0.2016, -0.1380,  ...,  0.8497,  0.1098, -0.4725])
tensor([ 0.4408, -0.8650,  0.4185,  ...,  1.8724,  0.9284, -1.0298])
tensor([-0.2379, -0.2091, -0.2344,  ...,  1.4266,  2.5727, -2.2658])
tensor([-0.2543,  0.0933,  0.0835,  ..., -1.2780,  0.1828,  0.1095])
tensor([-0.0690,  0.1126,  0.2371,

tensor([-0.0105, -0.5830,  0.2711,  ...,  0.4709, -0.4254,  0.0409])
tensor([-0.4959,  0.2016,  0.2681,  ...,  1.8740, -1.0130, -0.5823])
tensor([ 0.1915,  0.2300, -0.2894,  ...,  0.5149,  0.2934,  0.6817])
tensor([-0.1362,  0.7685, -0.4232,  ...,  0.4051, -0.5549, -0.2037])
tensor([-0.2569,  0.4148, -0.4112,  ..., -0.8821, -0.0417, -0.0718])
tensor([-0.0690,  0.1126,  0.2371,  ...,  0.3418,  1.9120, -0.3926])
tensor([ 0.0118, -0.9522,  0.5399,  ...,  0.8009,  1.8252,  0.3621])
tensor([ 0.1131,  0.1575,  0.1414,  ..., -0.0573,  2.5712,  0.6763])
tensor([ 0.2248, -0.7805,  1.2900,  ..., -0.1038,  1.7571,  0.3026])
tensor([0.1131, 0.1575, 0.1414,  ..., 0.1078, 2.8473, 1.6397])
tensor([0.4781, 0.0239, 0.3786,  ..., 0.0743, 2.1912, 0.7565])
tensor([-0.1052, -0.9387, -0.0376,  ...,  0.4522,  3.0771,  0.6239])
tensor([0.1131, 0.1575, 0.1414,  ..., 0.2567, 1.8569, 1.1424])
tensor([ 0.5007, -0.8363,  0.0597,  ..., -0.5585,  2.3980,  0.8011])
tensor([-0.1052, -0.9387, -0.0376,  ...,  0.3251,  2

In [120]:
from flair.embeddings import ELMoEmbeddings
from flair.data import Sentence
from tqdm import tqdm
import numpy as np


class Elmo(object):
    def __init__(self):
        # init embedding
        self.embedding = ELMoEmbeddings()

    def fit_transform(self, X):
        elmo_list = []
        for line in tqdm(X):
            dict_emb = {}
            sentence = Sentence(line)
            self.embedding.embed(sentence)
            for token in sentence:
                dict_emb[token] = token.embedding
            elmo_list.append(dict_emb)
        return np.array(elmo_list)


In [121]:
el = Elmo()
output = el.fit_transform(x)

100%|██████████| 100/100 [01:36<00:00,  1.04it/s]


In [124]:
output

array([{Token[0]: "The": tensor([-0.3288,  0.2022, -0.5940,  ..., -0.2876,  0.4907,  0.0943]), Token[1]: "presence": tensor([-0.4335,  1.0174, -0.7131,  ..., -0.3750,  1.4164, -0.4077]), Token[2]: "of": tensor([ 0.2478, -0.2490,  0.0222,  ..., -1.6505, -0.4505, -0.1460]), Token[3]: "communication": tensor([ 0.3358, -0.6698, -0.0683,  ...,  0.5426,  0.2742, -0.2719]), Token[4]: "amid": tensor([-5.6628e-01, -2.1065e-02,  2.1309e-06,  ..., -4.3520e-01,
                7.5123e-01, -5.5567e-01]), Token[5]: "scientific": tensor([-0.3897, -0.5744,  0.1275,  ...,  1.6141,  0.3421, -1.2750]), Token[6]: "minds": tensor([-0.4204, -0.1053, -0.7463,  ...,  1.6170,  1.7664, -0.2729]), Token[7]: "was": tensor([ 0.2981, -0.0738, -0.1319,  ...,  0.3482,  0.7202,  0.2195]), Token[8]: "equally": tensor([-0.1151, -0.6863, -0.0091,  ...,  0.2671,  0.0370, -1.0262]), Token[9]: "important": tensor([-0.4031, -0.2223, -0.0422,  ..., -0.1547,  0.9290, -1.7621]), Token[10]: "to": tensor([-0.1821,  0.0381,  0.169

In [108]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-cloud -y
!pip install -U tensorflow==1.15

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\cli\base_command.py", line 180, in _main
    status = self.run(options, args)
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\commands\uninstall.py", line 89, in run
    uninstall_pathset.commit()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\req\req_uninstall.py", line 442, in commit
    self._moved_paths.commit()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\req\req_uninstall.py", line 282, in commit
    save_dir.cleanup()
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_internal\utils\temp_dir.py", line 184, in cleanup
    rmtree(self._path)
  File "C:\Users\D073999\Documents\Priv-Dev\Retriever\lib\site-packages\pip\_vendor\tenacity\__init__.py", line 339, in wrapped_f
    return self(f, *args, **kw)
  

Looking in indexes: https://int.repositories.cloud.sap/artifactory/api/pypi/build-releases-pypi/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/build-milestones-pypi/simple
  Using cached https://int.repositories.cloud.sap/artifactory/api/pypi/build-milestones-pypi/packages/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=2e79851a5a64d77ddea493df99b35153515a67e6f636e2315774d102362af3c8
  Stored in directory: c:\users\d073999\appdata\local\pip\cache\wheels\72\f2\b3\e50dc833b3e9829b60168ad4921ee7ccbfffe6738204b06a38
Successfully built gast
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Suc

You should consider upgrading via the 'C:\Users\D073999\Documents\Priv-Dev\Retriever\Scripts\python.exe -m pip install --upgrade pip' command.


In [109]:
!pip uninstall pytorch-lightning -y
!pip uninstall tensorflow-probability -y

In [154]:
##Tensorflow Hub ELMO-2
import tensorflow_hub as hub
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

elmo = hub.load("https://tfhub.dev/google/elmo/2").signatures["default"]

def create_elmo_embeddings(data):
    embed=elmo(
        tf.constant(data))["elmo"]
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        sess.run(tf.compat.v1.tables_initializer())
        out_x=sess.run(embed)
        #out_y=ses.run(tf.reduce_mean(embed,1))
        return out_x
z=queries.Query.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [228]:
x = collection.Passage[:40]

In [229]:
x

0     The presence of communication amid scientific ...
1     The Manhattan Project and its atomic bomb help...
2     Essay on The Manhattan Project - The Manhattan...
3     The Manhattan Project was the name for a proje...
4     versions of each volume as well as complementa...
5     The Manhattan Project. This once classified ph...
6     Nor will it attempt to substitute for the extr...
7     Manhattan Project. The Manhattan Project was a...
8     In June 1942, the United States Army Corps of ...
9     One of the main reasons Hanford was selected a...
10    group discussions, community boards or panels ...
11    punishment designed to repair the damage done ...
12    Tutorial: Introduction to Restorative Justice....
13    Organize volunteer community panels, boards, o...
14    The purpose of this paper is to point out a nu...
15    Each of these types of communitiesâthe geogr...
16    The approach is based on a theory of justice t...
17    Inherent in many peopleâs understanding 

In [172]:
z=x.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [187]:
def elmo_vectors(x):
  embeddings = []
  for elm in x:
        print(elm)
        e = elmo(elm.tolist())["elmo"]
        embeddings.append(elmo(x.tolist())["elmo"])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [230]:
x = x.progress_apply(lambda text: np.array(
            stemming(
                removal(
                    tokenization(text)
                ))));x

100%|██████████| 40/40 [00:00<00:00, 367.86it/s]


0     [presenc, commun, amid, scientif, mind, equal,...
1     [manhattan, project, atom, bomb, help, bring, ...
2     [essay, manhattan, project, manhattan, project...
3     [manhattan, project, name, project, conduct, w...
4     [version, volum, well, complementari, websit, ...
5     [manhattan, project, classifi, photograph, fea...
6     [attempt, substitut, extraordinarili, rich, li...
7     [manhattan, project, manhattan, project, resea...
8     [june, 1942, unit, state, armi, corp, engineer...
9     [one, main, reason, hanford, select, site, man...
10    [group, discuss, commun, board, panel, third, ...
11    [punish, design, repair, damag, done, victim, ...
12    [tutori, introduct, restor, justic, restor, ju...
13    [organ, volunt, commun, panel, board, committe...
14    [purpos, paper, point, number, unresolv, issu,...
15    [type, communitiesâth, geograph, commun, vic...
16    [approach, base, theori, justic, consid, crime...
17    [inher, mani, peopleâ, understand, notio

In [244]:
output = []
from nltk.tokenize.treebank import TreebankWordDetokenizer
output = []
for tokens in x:
    r =TreebankWordDetokenizer().detokenize(tokens)
    output.append(r)
output

['presenc commun amid scientif mind equal import success manhattan project scientif intellect cloud hang impress achiev atom research engin success truli meant hundr thousand innoc live obliter',
 'manhattan project atom bomb help bring end world war ii legaci peac use atom energi continu impact histori scienc',
 'essay manhattan project manhattan project manhattan project see make atom bomb possibl success project would forev chang world forev make known someth power manmad',
 'manhattan project name project conduct world war ii develop first atom bomb refer specif period project 194 â\x80¦ 21946 control us armi corp engin administr gener lesli r grove',
 'version volum well complementari websit first websiteâ\x80\x93th manhattan project interact historyâ\x80\x93i avail offic histori heritag resourc websit http wwwcfo doegovme70histori offic histori heritag resourc nation nuclear secur',
 'manhattan project classifi photograph featur first atom bomb â\x80\x94 weapon atom scientist nic

In [245]:
z=output#.tolist()
elmo_input=z[:2]
elmo_output=create_elmo_embeddings(elmo_input)

In [247]:
elmo_output

array([[[-0.16142987,  0.39164305,  0.2924859 , ..., -0.10307619,
          0.47838593,  0.10405383],
        [ 0.103149  , -0.20897631,  0.40901518, ..., -0.14926252,
          0.6624829 , -0.18022116],
        [-0.24741618,  0.30307624, -0.5055893 , ..., -0.26527908,
          0.17358826, -0.03860291],
        ...,
        [-0.12553898, -0.13239129, -0.04669396, ..., -0.32613608,
          0.5977138 ,  0.13836601],
        [ 0.5781988 , -0.35236633, -0.37354636, ..., -0.38017738,
          0.07846026,  0.09849334],
        [ 0.1008294 , -0.49969184, -0.3491489 , ...,  0.12266825,
          0.16703467,  0.42627513]],

       [[ 0.306329  ,  0.4091187 ,  0.48765004, ...,  0.13162018,
          0.62339723, -0.22805919],
        [ 0.7706266 , -0.03343945,  0.12615913, ...,  0.03431229,
          0.20650187, -0.33816066],
        [-0.01331159,  0.18552707, -0.15152037, ...,  0.21959701,
          0.87772703,  0.6102616 ],
        ...,
        [-0.02840841, -0.04353216,  0.04130162, ...,  

elmo_output